# Entity VO DTO 의 차이점
## 서론
프로그래밍을 시작하면서 위의 세 개를 혼동하기 쉽습니다. 사실 개발자들 끼리 정한 이름이고 프로그램에서 개발자가 설계한 기능만 정상 작동되면 상관없습니다. 하지만 이 세 개의 개념에 대한 이론을 제대로 알고 있어야 실무나 협업 프로젝트를 진행함에 있어서 수월하게 이해하실 수 있을겁니다.  

&& Entity VO DTO 의 개념을 쉽게 설명하기 위해 JPA를 적용하여 설명할 것임


---

## 1. Entity

Entity(이하 엔티티)는 DTO VO 보다 DB에 근접하여 정의되어진 클래스라고 볼 수 있습니다. 엔티티 클래스는 상속받거나 구현체로 존재해서는 안되며 DB 테이블 내에 존재하지 않는 컬럼을 가져서는 안됩니다.

> RDB(Relational DataBase, 관계형 데이터베이스)에서의 Entity(개체)란, 현실세계에서의 개체를 표현하기 위한 유형, 무형의 실체로써, Entity를 표현하기 위해서 테이블을 생성합니다.

### 1.1 Entity의 실무 사용
사용처마다 다르겠지만 domain 패키지 안에 **[DB에 선언된 테이블 명 = 엔티티]** 클래스를 만들고 엔티티의 속성들을 선언해줍니다.  
또한 JPA를 사용한다면 domain 패키지 안에 엔티티마다 Repository 인터페이스를 만들어주어 JPA 쿼리를 사용합니다.  

추신, Mybatis를 사용하는 경우 Entity VO DTO 를 정확히 구분해서 개발하는게 생산성이 더 떨어짐!!

#### domain.엔티티 클래스
```java
@Getter
@Setter
@Entity
@Table(name = "admin")
@Where(clause = "delete_yn = 'N'")
public class Admin extends BaseDomain implements DomainWithMapper, Serializable {

    @Id
    @Column(nullable = false, unique = true)
    private Long adminId;   //사용자 id

    @Column(nullable = false, length = 15)
    private String role;

    @Column(nullable = false, length = 50)
    private String adminName;

    @Column(nullable = false, length = 50)
    private String slackId;

    @Column(nullable = false, length = 256)
    private String adminPassword;

    @Column(length = 15)
    private String phone;

    @Column(nullable = false, length = 1)
    private String status;


    @Column(nullable = false, length = 512)
    private String profileImg;

    @LastModifiedDate
    @Column(nullable = false)
    private LocalDate updateAt;

    @LastModifiedBy
    @Column(nullable = false, length = 50)
    private String updateBy;

    @Column(nullable = false, length = 1)
    private String deleteYn;

    @Override
    public AdminDto.Response toResponse() {
        return AdminDtoMapper.INSTANCE.entityToResponse(this);
    }
}

```
---

#### domain.엔티티Repository 인터페이스
```java
public interface SettlementTotalRepository extends BaseRepository<SettlementTotal> {

  @Query("SELECT SUM(depositExpectedAmount) FROM SettlementTotal WHERE pgId = :pgId AND settlementedAt = :settlementedAt")
  Long getDepositExpectedAmount(@Param("pgId") String pgId,
      @Param("settlementedAt") LocalDate settlementedAt);

  @Query("SELECT st.settlementedAt from SettlementTotal as st "
      + "WHERE (st.settlementStatus <> 'confirmed' OR st.settlementStatus is NULL) "
      + "AND st.settlementedAt < :settlementedAt "
      + "GROUP BY st.settlementedAt")
  List<LocalDate> getNotRunningDate(@Param("settlementedAt") LocalDate settlementedAt);

  @Query("SELECT distinct settlementStatus FROM SettlementTotal WHERE pgId = :pgId AND settlementedAt = :settlementedAt")
  String getSettlementStatus(@Param("pgId") String pgId,
      @Param("settlementedAt") LocalDate settlementedAt);
}
```

<hr>

## 2. VO (Value Object)

VO 는 값을 가진 객체로 equals()와 hashcode()를 오버라이딩하는 핵심역할을 가지고 있다.  
VO 내부에 선언된 속성(필드)의 모든 값들이 VO 객체마다 값이 같아야, 똑같은 객체라고 판별한다.
> VO는 Getter와 Setter를 가질 수 있으며, VO는 테이블 내에 있는 속성 외에 추가적인 속성을 가질 수 있으며, 여러 테이블(A, B, C)에 대한 공통 속성을 모아서 만든 BaseVO 클래스를 상속받아서 사용할 수도 있다.

---

## 3. DTO(Data Transfer Object)
DTO(Data Transfer Object)는 데이터 전송(이동) 객체라는 의미를 가지고 있다. DTO는 주로 비동기 처리를 할 때 사용한다.
> 계층간 데이터 교환을 위한 객체(Java Beans)이다.
DB의 데이터를 Service나 Controller 등으로 보낼 때 사용하는 객체를 말한다.
즉, DB의 데이터가 Presentation Logic Tier로 넘어올때는 DTO로 변환되어 오고가는 것이다.
로직을 갖고 있지 않는 순수한 데이터 객체이며, getter/setter 메서드만을 갖는다.
또한 Controller Layer에서 Response DTO 형태로 Client에 전달한다.

### 3.1 DTO 의 실무 사용
구성은 dto 패키지 안에 Dto와 DtoMapper로 이루어진 구조가 있다. 가장 먼저 Dto의 구성을 보자면 Dto는 앞서 설명한 것과 같이 계층간 데이터 교환을 위한 객체이다. 따라서 해당 클래스는 DB의 데이터를 Service나 Controller 등으로 보낼 때 사용되기 때문에 요청과 응답으로 구성되면 편하다. 다음 코드는 실무에서 사용되는 구조이다.

#### dto.엔티티Dto 클래스
```java
public class AdminDto {

    @Data
    @ApiModel("관리자계정관리 조회")
    public static class Request  extends PagableDto.Request{

        @DateTimeFormat(pattern = "yyyy-MM-dd")
        private LocalDate createAtStart;

        @DateTimeFormat(pattern = "yyyy-MM-dd")
        private LocalDate createAtEnd;

        private String status;

        private String role;

        private String keywordType;

        private String keyword;
    }

    @Data
    @ApiModel("관리자계정관리 조회 정보")
    public static class Response {
        @ApiModelProperty(value = "관리자 번호", position = 1)
        private Long adminId;

        @ApiModelProperty(value = "권한그룹", position = 2)
        private String role;

        @ApiModelProperty(value = "관리자 이름", position = 3)
        private String adminName;

        @ApiModelProperty(value = "관리자 아이디", position = 4)
        private String slackId;

        @ApiModelProperty(value = "관리자 비밀번호", position = 5)
        private String adminPassword;

        @ApiModelProperty(value = "관리자 전화번호", position = 6)
        private String phone;

        @ApiModelProperty(value = "관리자 상태", position = 7)
        private String status;

        @ApiModelProperty(value = "관리자 계정 생성일시", position = 8)
        private LocalDateTime createAt;

        @ApiModelProperty(value = "관리자 계정 수정일시", position = 9)
        private LocalDateTime updateAt;

        @ApiModelProperty(value = "관리자 계정 수정자", position = 10)
        private String updateBy;

        @ApiModelProperty(value = "관리자 삭제 여부", position = 11)
        private String deleteYn;

    }
}

```

#### dto.엔티티DtoMapper 클래스
```java
@Mapper
public interface AdminDtoMapper extends BaseDtoMapper {

    AdminDtoMapper INSTANCE = Mappers.getMapper(AdminDtoMapper.class);

    AdminDto.Response entityToResponse(Admin admin);

}
```


+ 1. AdminDtoMapper INSTANCE = Mappers.getMapper(AdminDtoMapper.class);  

해당 문장은 싱글톤과 유사한 형태를 지닌다. 해당 매퍼를 서비스나 컨트롤단에서 다음과 같이 사용할 수 있다.  
```java
AdminDtoMapper.INSTANCE.entityToResponse(v)
```
매퍼의 역할은 DB의 영역 값(엔티티)들을 Dto의 영역으로 끌어들여 사용할 수 있게 만들어주는 것이다.   



+ 2. AdminDto.Response entityToResponse(Admin admin);  

해당 문장이 앞서 설명한 엔티티를 DTO의 형태로 바꾸는 문장이다 다음의 문장은 엔티티를 개발자가 정의한 Response 로 전환해주는 문장이다. 이런 형태를 취하면 불필요하게 엔티티의 모든 값을 다룰 필요가 없으며 개발자가 정의한(사용자가 원하는) 요청의 값들만 Response하면 되는 것이다.
```java
AdminDto.Response entityToResponse(Admin admin);
```


> DTO와 Entity간의 변환에 대해 자세히 알고 싶다면 MapStruct 문서 참조